Import libraries for data cleaning 

In [1]:
import csv
from tokenize import String

from db_connection import get_db_engine
from db_connection import execute_query
import numpy as np
import pandas as pd
import seaborn as sns
import os
import re

Connection Successful!


Reading CSV file

In [2]:
df_genres = pd.read_csv(r"C:\Users\aless\Desktop\Uni\Dataset\genres.csv\genres.csv")

Top records of CSV file: genres.csv

In [3]:
df_genres

,id,genre
0,1000001,Comedy
1,1000001,Adventure
2,1000002,Comedy
3,1000002,Thriller
4,1000002,Drama
...,...,...
1046844,1941563,Drama
1046845,1941566,Crime
1046846,1941569,Crime
1046847,1941596,Action


Checking for total dimension of the csv

In [4]:
df_genres.shape

(1046849, 2)

Rename 'id' column in 'id_genre'. "inplace = True" modifies directly the df without creating a copy

In [5]:
df_genres.rename(columns = {"id":"id_movie"} , inplace = True)


In [6]:
df_genres.columns

Index(['id_movie', 'genre'], dtype='object')

Check info about column type

In [7]:
df_genres.dtypes

id_movie     int64
genre       object
dtype: object

Correction of data type of the records

In [8]:
df_genres['id_movie'] = df_genres['id_movie'].astype('Int64')
df_genres['genre'] = df_genres['genre'].fillna(pd.NA).astype('string')

df_genres.dtypes

id_movie             Int64
genre       string[python]
dtype: object

Check if there are duplicates

In [9]:
df_genres.duplicated().sum()


np.int64(0)

In [10]:
df_genres.drop_duplicates(inplace = True)
df_genres.duplicated().sum()

np.int64(0)

Check the missing values

In [11]:
df_genres.isnull().sum()

id_movie    0
genre       0
dtype: int64

Dropping records with missing value 

In [12]:
df_genres = df_genres.dropna()
df_genres.isnull().sum()

id_movie    0
genre       0
dtype: int64

Remove genres where id_movie doesn't exist

In [13]:
valid_movies = pd.read_csv("movies_valid_ids.csv")
df_genres = df_genres[df_genres["id_movie"].isin(valid_movies["id_movie"])]
df_genres.shape

(1046845, 2)

Connecting to PostgreSQL database and creating the table

In [14]:
engine = get_db_engine()
sql = \
    """CREATE TABLE IF NOT EXISTS genres (
            id BIGSERIAL PRIMARY KEY,
            id_movie INTEGER CHECK (id_movie>0),
            genre TEXT CHECK (genre<>'')
            );
    """
execute_query(sql)

Query Executed Successfully!


Population of the table using .to_sql function

In [15]:
df_genres.to_sql("genres", engine, if_exists="append", index=False)

845